In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [3]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

In [2]:
#Get the html from the wikipedia page
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
#Create a BeautifulSoup object
soup=BeautifulSoup(source,'lxml')

In [4]:
#Find the data to be scraped
table=soup.find('table',class_='wikitable sortable')
print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [5]:
items= table.find('tbody')
heading =items.text
print(items)

<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" tit

In [7]:
#Define column names for the datarame
column_names=['PostalCode','Borough','Neighborhood']
#Create an empty list
data=[]
i=0
#Extract the data from the table and populate the list
for item in items.findAll('td'):
    item=item.text
    data.append(item.strip())
#    print(list_[i])
    if i<len(items.find_all('td')):
        i+=1
print(len(data))
print(data)


867
['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned', 'M3A', 'North York', 'Parkwoods', 'M4A', 'North York', 'Victoria Village', 'M5A', 'Downtown Toronto', 'Harbourfront', 'M5A', 'Downtown Toronto', 'Regent Park', 'M6A', 'North York', 'Lawrence Heights', 'M6A', 'North York', 'Lawrence Manor', 'M7A', "Queen's Park", 'Not assigned', 'M8A', 'Not assigned', 'Not assigned', 'M9A', 'Etobicoke', 'Islington Avenue', 'M1B', 'Scarborough', 'Rouge', 'M1B', 'Scarborough', 'Malvern', 'M2B', 'Not assigned', 'Not assigned', 'M3B', 'North York', 'Don Mills North', 'M4B', 'East York', 'Woodbine Gardens', 'M4B', 'East York', 'Parkview Hill', 'M5B', 'Downtown Toronto', 'Ryerson', 'M5B', 'Downtown Toronto', 'Garden District', 'M6B', 'North York', 'Glencairn', 'M7B', 'Not assigned', 'Not assigned', 'M8B', 'Not assigned', 'Not assigned', 'M9B', 'Etobicoke', 'Cloverdale', 'M9B', 'Etobicoke', 'Islington', 'M9B', 'Etobicoke', 'Martin Grove', 'M9B', 'Etobicoke', 'Princess Gardens', 

In [10]:
#Drop the rows with 'Not assigned' Borough values and rest the index
neighborhood=neighborhood[neighborhood.Borough!='Not assigned']
neighborhood.reset_index(inplace=True)
neighborhood.head(5)

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


In [9]:
#Create a datarame based on the list, with the column names
neighborhood=pd.DataFrame(np.array(data).reshape(289,3),columns=column_names)
neighborhood.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
del neighborhood['index']

In [12]:
#Assign the borough value to the 'Not assigned' neighborhood value
neighborhood.at[neighborhood.index[6],'Neighborhood']=neighborhood.at[neighborhood.index[6],'Borough']
neighborhood.at[neighborhood.index[6],'Neighborhood']

"Queen's Park"

In [13]:
neighborhood.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [20]:
list_=neighborhood['Neighborhood']
#def func():
    
neighborhood=neighborhood.groupby(('PostalCode'),as_index=False).agg(lambda x:','.join(set(x))).reset_index()
neighborhood.head()

,index,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern,Rouge"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


In [21]:
del neighborhood['index']

In [22]:
neighborhood.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
neighborhood.shape

(103, 3)